# Homework 2: Unstructured Data
## Joe Morris, Amrit Sandhu, Aidan Cremins, Ben Kanarick, Mark Moreno - 10:30 Section

In [1]:
# imports
import pandas as pd
import numpy as np
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import spacy 

/Users/joemorris/opt/anaconda3/lib/python3.8/site-packages/h5py/__init__.py:72: UserWarning: h5py is running against HDF5 1.10.4 when it was built against 1.8.4, this may cause problems
  _warn(("h5py is running against HDF5 {0} when it was built against {1}, "


### Task A. Extract about 5-6k reviews. 

##### NOTE: Only 2216 reviews with text were aviailable to scrape

In [2]:
# read in the beer_reviews.csv file
df = pd.read_csv('beer_reviews.csv')
print('Length of the dataframe: ', len(df))
df.head()

Length of the dataframe:  2216


,Beer,Review,Rating
0,Kentucky Brunch Brand Stout,I didnt think i was going to give it a perfect...,5.00
1,Kentucky Brunch Brand Stout,So I just read a review that called the legend...,3.79
2,Kentucky Brunch Brand Stout,"2021 vintage, bottle #79\n\nHoly. Fucking. Shi...",4.64
3,Kentucky Brunch Brand Stout,"Celebrating my buddy @Rug with his 1,000th bee...",4.27
4,Kentucky Brunch Brand Stout,"Thick and syrupy pour, mocha head. Aroma is bo...",4.79


In [3]:
# Clean Messages
df['clean_review'] = df['Review'].str.replace('\...', ' ') # remove ellipses
df['clean_review'] = df['clean_review'].str.replace('\n', ' ') # remove newlines
df['clean_review'] = df['clean_review'].str.replace("'", '') # remove apostrophes
df['clean_review'] = df['clean_review'].str.replace('[^\w\s]',' ') # remove punctuation
df['clean_review'] = df['clean_review'].str.replace('\d+', '') # remove numbers
df['clean_review'] = df['clean_review'].str.lower() # make lowercase

# remove stop words from clean review nltk
from nltk.corpus import stopwords
stop = stopwords.words('english')
df['clean_review'] = df['clean_review'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))


df = df.dropna(subset=['clean_review'])  # remove nan from message column
df = df.reset_index(drop=True)
df.head()

,Beer,Review,Rating,clean_review
0,Kentucky Brunch Brand Stout,I didnt think i was going to give it a perfect...,5.00,didnt think going give perfect score looks sme...
1,Kentucky Brunch Brand Stout,So I just read a review that called the legend...,3.79,read review called legendary kbbs typical whit...
2,Kentucky Brunch Brand Stout,"2021 vintage, bottle #79\n\nHoly. Fucking. Shi...",4.64,vintage bottle holy ucking hit bbs eer review ...
3,Kentucky Brunch Brand Stout,"Celebrating my buddy @Rug with his 1,000th bee...",4.27,celebrating buddy rug th beer review almost pe...
4,Kentucky Brunch Brand Stout,"Thick and syrupy pour, mocha head. Aroma is bo...",4.79,thick syrupy pour mocha head roma boozy maple ...


### Task B. Assume that a customer, who will be using this recommender system, has specified 3 attributes in a product. E.g., one website describes multiple attributes of beer:

https://www.dummies.com/food-drink/drinks/beer/beer-for-dummies-cheat-sheet/

<li>Aggressive (Boldly assertive aroma and/or taste) </li>
<li>Balanced: Malt and hops in similar proportions; equal representation of malt sweetness and hop bitterness in the flavor — especially at the finish </li>
<li>Complex: Multidimensional; many flavors and sensations on the palate </li>
<li>Crisp: Highly carbonated; effervescent </li>
<li>Fruity: Flavors reminiscent of various fruits or Hoppy: Herbal, earthy, spicy, or citric aromas and flavors of hops or Malty: Grainy, caramel-like; can be sweet or dry </li>
<li>Robust: Rich and full-bodied</li>

##### Use the above attributes as examples only, for a word frequency analysis of beer reviews is a better way to find important attributes in the actual data. 
##### Assume that a customer has specified three attributes of the product as being important to him or her. 

In [4]:
word_freq_all = pd.Series(' '.join(df['clean_review']).split()).value_counts()
word_freq_all[:10]

beer         1524
head         1164
taste         908
chocolate     875
dark          795
like          744
notes         705
bourbon       696
coffee        678
sweet         654
dtype: int64

### 3 Attributes: dark, sweet, head --> added to attributes.txt

### Task C. Perform a similarity analysis using cosine similarity (without word embeddings) with the 3 attributes specified by the customer and the reviews. 
##### The similarity script should accept as input a file with the product attributes, and calculate similarity scores (between 0 and 1) between these attributes and each review. That is, the output file should have 3 columns – product_name (for each product, the product_name will repeat as many times as there are reviews of the product), product_review and similarity_score. 

In [5]:
# read in attributes txt file
attributes = pd.read_csv('attributes.txt', header=None, names=['attribute'])
attributes.head()

# change attributes to a string
attributes = attributes['attribute'].str.cat(sep=' ').lower()

In [6]:
# get unique words in a specific review list and attributes list
def unique_words(review, attributes):
    unique = []
    for word in review:
        if word not in unique:
            unique.append(word)
    for word in attributes:
        if word not in unique:
            unique.append(word)
    return unique

# two vectors of word counts from reviews and attributes indexed by unique words
def word_count(review, attributes):
    review = review.split()
    attributes = attributes.split()
    unique = unique_words(review, attributes)
    review_count = []
    attributes_count = []
    for word in unique:
        review_count.append(review.count(word))
        attributes_count.append(attributes.count(word))
    return review_count, attributes_count

# create a function to calculate the cosine similarity between two vectors
def cosine_similarity(v1, v2):
    dot = np.dot(v1, v2)
    norma = np.linalg.norm(v1)
    normb = np.linalg.norm(v2)
    return dot / (norma * normb)

df['similarity'] = df.apply(lambda x: cosine_similarity(word_count(x['clean_review'], attributes)[0], word_count(x['clean_review'], attributes)[1]), axis=1)

# create new dataframe with Beer, Review, and similarity score
df2 = df[['Beer', 'Review', 'similarity']]
df2.columns = ['product_name', 'product_review', 'similarity_score']
df2.head()


/var/folders/yh/03ygvdwj1hb1wmtsp43j_w1c0000gn/T/ipykernel_57962/31485790.py:29: RuntimeWarning: invalid value encountered in double_scalars
  return dot / (norma * normb)


,product_name,product_review,similarity_score
0,Kentucky Brunch Brand Stout,I didnt think i was going to give it a perfect...,0.000000
1,Kentucky Brunch Brand Stout,So I just read a review that called the legend...,0.029501
2,Kentucky Brunch Brand Stout,"2021 vintage, bottle #79\n\nHoly. Fucking. Shi...",0.032275
3,Kentucky Brunch Brand Stout,"Celebrating my buddy @Rug with his 1,000th bee...",0.068519
4,Kentucky Brunch Brand Stout,"Thick and syrupy pour, mocha head. Aroma is bo...",0.107211


### Task D. For every review, perform a sentiment analysis (using VADER). 

##### In case you have to change the default values of words in the VADER lexicon, use this article: https://medium.com/swlh/adding-ontext-to-unsupervised-sentiment-analysis-7b6693d2c9f8 

In [7]:
# clean product_review column
df2['product_review'] = df2['product_review'].str.replace('\n', ' ') # remove newlines

sent = SentimentIntensityAnalyzer()
df2['sentiment'] = df2['product_review'].apply(lambda x: sent.polarity_scores(x)['compound'])
df2.head()


/var/folders/yh/03ygvdwj1hb1wmtsp43j_w1c0000gn/T/ipykernel_57962/2304451302.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['product_review'] = df2['product_review'].str.replace('\n', ' ') # remove newlines
/var/folders/yh/03ygvdwj1hb1wmtsp43j_w1c0000gn/T/ipykernel_57962/2304451302.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['sentiment'] = df2['product_review'].apply(lambda x: sent.polarity_scores(x)['compound'])


,product_name,product_review,similarity_score,sentiment
0,Kentucky Brunch Brand Stout,I didnt think i was going to give it a perfect...,0.000000,0.9460
1,Kentucky Brunch Brand Stout,So I just read a review that called the legend...,0.029501,0.9976
2,Kentucky Brunch Brand Stout,"2021 vintage, bottle #79 Holy. Fucking. Shit....",0.032275,0.9927
3,Kentucky Brunch Brand Stout,"Celebrating my buddy @Rug with his 1,000th bee...",0.068519,0.9476
4,Kentucky Brunch Brand Stout,"Thick and syrupy pour, mocha head. Aroma is bo...",0.107211,0.8357


### Task E. Create an evaluation score for each beer that uses both similarity and sentiment scores. 

##### E.g., total score  = average of (similarity score + sentiment score) or a multiplicative model.  

##### Now recommend 3 products to the customer. 

In [8]:
# Multiplicative scores
df2['score'] = df2['similarity_score'] * df2['sentiment']

# new columns in original dataframe to use in other functions
df['score'] = df2['score']
df['sentiment'] = df2['sentiment']

# get average of score per beer and sort by highest score
df3 = df2.groupby('product_name')[['similarity_score', 'sentiment', 'score']].mean().sort_values(by=['score'], ascending=False)
df3.head(3)

,similarity_score,sentiment,score
product_name,,,
All That Is And All That Ever Will Be,0.265658,0.792720,0.210366
Stickee Monkee,0.229424,0.760656,0.175611
Double Shot,0.168706,0.871083,0.157164


##### Print out top 3 highest scored reviews for top recommended product to make sure that the reviews are relevant to the product attributes

In [9]:
# print top 3 product review and score for a specific beer sort by highest score
def top_3_reviews(beer):
    df4 = df2[df2['product_name'] == beer].sort_values(by=['score'], ascending=False)
    for i in range(3):
        print('Review:', df4.iloc[i]['product_review'])
        print('Score:', df4.iloc[i]['score'])
        print()

print('BEER:', df3.index[0], '\n')
top_3_reviews(df3.index[0])



BEER: All That Is And All That Ever Will Be 

Review: Pours like syrup--smooth and sweet. Very dark brown with a mocha head. It is rich with strong milk chocolate flavor, and subtle coffee flavor underneath. Several good aspects to this stout, but perhaps a bit too sweet, making this lean toward dessert beer.
Score: 0.35585220259995315

Review: 16oz can poured into a snifter.  A: Black on the pour with two fingers of brown head. Retention is good, lacing manifests in broken rings.  S: Nose is heavy on the chocolate. Brownie batter and a generous helping of Hershey’s chocolate syrup. Fatty nibs. Bit of woody roast and earthy hop. Berry-like fruity note. Nutty coffee.  T: Follows the nose. Chocolate all over the attack. Cocoa nibs, brownie batter, chocolate syrup in a big way. Sugary lactose. That berry fruitiness comes through tasting like a raspberry tootsie pop. Pervasive background flavor of nutty coffee. Quick flash of earthy hops, and woody and roasty malt flavors poke through now 

### Task F. How would your recommendation change if you use word vectors (e.g., the spaCy package with medium sized pretrained word vectors) instead of the plain vanilla bag-of-words cosine similarity? 

##### One way to analyze the difference would be to consider the % of reviews that mention a preferred attribute. E.g., if you recommend a product, what % of its reviews mention an attribute specified by the customer? Do you see any difference across bag-of-words and word vector approaches? 

In [10]:
attributes = attributes.split()

df['contains_{}'.format(attributes[0])] = df['clean_review'].str.contains(attributes[0])
df['contains_{}'.format(attributes[1])] = df['clean_review'].str.contains(attributes[1])
df['contains_{}'.format(attributes[2])] = df['clean_review'].str.contains(attributes[2])

# ratings column used to get the count of reviews for each unique beer
subset = df.groupby('Beer').agg({'score': 'mean', 'contains_{}'.format(attributes[0]): 'sum', 'contains_{}'.format(attributes[1]): 'sum', 'contains_{}'.format(attributes[2]): 'sum','Rating':'count'}).reset_index()
subset['contains_{}'.format(attributes[0])] = subset['contains_{}'.format(attributes[0])] / subset['Rating']
subset['contains_{}'.format(attributes[1])] = subset['contains_{}'.format(attributes[1])] / subset['Rating']
subset['contains_{}'.format(attributes[2])] = subset['contains_{}'.format(attributes[2])] / subset['Rating']
subset = subset.drop(columns=['Rating'])
subset = subset.sort_values(by=['score'], ascending=False)[:3]
subset

,Beer,score,contains_dark,contains_sweet,contains_head
11,All That Is And All That Ever Will Be,0.210366,1.000000,0.6,0.800000
211,Stickee Monkee,0.175611,0.333333,1.0,0.666667
85,Double Shot,0.157164,0.666667,0.5,0.666667


In [ ]:
nlp = spacy.load("en_core_web_md")
attributes_word_vec = nlp(attributes[0] + ' ' + attributes[1] + ' ' + attributes[2]).vector

# add spacy_similarity_score to df dataframe
# iterate over every rw in df dataframe
for index, row in df.iterrows():
    try :
        # get the word vector for the review
        review_word_vec = nlp(row['clean_review']).vector
        # calculate the similarity between the review and the attributes
        similarity = review_word_vec.similarity(attributes_word_vec)
        # add the similarity score to the df dataframe
        df.loc[index, 'spacy_similarity_score'] = similarity
    except:
        # if there is an error, add a nan to the df dataframe
        df.loc[index, 'spacy_similarity_score'] = np.nan
#df['spacy_similarity_score'] = df['clean_review'].apply(lambda x: nlp(x).similarity(attributes_word_vec))


In [ ]:
df.head()

In [ ]:
# print review, spacy_similarity_score, and score for df dataframe

def top_3_reviews_spacy(beer):
    df4 = df[df['Beer'] == beer].sort_values(by=['spacy_similarity_score'], ascending=False)
    for i in range(5):
        print('Review:', df4.iloc[i]['Review'])
        print('Spacy Similarity Score:', df4.iloc[i]['spacy_similarity_score'])
        print('Score:', df4.iloc[i]['score'])
        print()

top_3_reviews_spacy('Kentucky Brunch Brand Stout')

In [ ]:
# print min and max of spacy_similarity_score for df dataframe
print('Min:', df['spacy_similarity_score'].min())
print('Max:', df['spacy_similarity_score'].max())

# print row with min spacy_similarity_score
df[df['spacy_similarity_score'] == df['spacy_similarity_score'].min()]

# print row with max spacy_similarity_score
df[df['spacy_similarity_score'] == df['spacy_similarity_score'].max()]['Review'][280]


In [ ]:
df['spacy_score'] = df['spacy_similarity_score'] * df['sentiment']
for beer in (df['Beer']).unique():
    df['contains_{}'.format(attributes[0])] = df['clean_review'].str.contains(attributes[0])
    df['contains_{}'.format(attributes[1])] = df['clean_review'].str.contains(attributes[1])
    df['contains_{}'.format(attributes[2])] = df['clean_review'].str.contains(attributes[2])

subset2 = df.groupby('Beer').agg({'spacy_score': 'mean', 'contains_{}'.format(attributes[0]): 'sum', 'contains_{}'.format(attributes[1]): 'sum', 'contains_{}'.format(attributes[2]): 'sum','Rating':'count'}).reset_index()
subset2['contains_{}'.format(attributes[0])] = subset2['contains_{}'.format(attributes[0])] / subset2['Rating']
subset2['contains_{}'.format(attributes[1])] = subset2['contains_{}'.format(attributes[1])] / subset2['Rating']
subset2['contains_{}'.format(attributes[2])] = subset2['contains_{}'.format(attributes[2])] / subset2['Rating']
subset2 = subset2.drop(columns=['Rating'])
subset2 = subset2.sort_values(by=['spacy_score'], ascending=False)[:3]
subset2

### Task G. How would your recommendations differ if you ignored the similarity and feature sentiment scores and simply chose the 3 highest rated products from your entire dataset? Would these products meet the requirements of the user looking for recommendations? Why or why not? 

##### Justify your answer with analysis. Use the similarity and sentiment scores as well as overall ratings to answer this question. 

In [ ]:
top_3_beers = df.groupby('Beer').agg({'Rating': 'mean'}).reset_index().sort_values(by='Rating',ascending=False)[:3]
top_3_beers_df = top_3_beers.merge(df, on='Beer', how='left')
subset3 = top_3_beers_df.groupby('Beer').agg({'score': 'mean', 'contains_{}'.format(attributes[0]): 'sum', 'contains_{}'.format(attributes[1]): 'sum', 'contains_{}'.format(attributes[2]): 'sum','Rating_x':'count'}).reset_index()
subset3['contains_{}'.format(attributes[0])] = subset3['contains_{}'.format(attributes[0])] / subset3['Rating_x']
subset3['contains_{}'.format(attributes[1])] = subset3['contains_{}'.format(attributes[1])] / subset3['Rating_x']
subset3['contains_{}'.format(attributes[2])] = subset3['contains_{}'.format(attributes[2])] / subset3['Rating_x']
subset3 = subset3.drop(columns=['Rating_x'])
subset3 = subset3.sort_values(by=['score'], ascending=False)[:3]
subset3.reset_index(drop=True)

### Task H. Using the top four attributes of beer (from word frequency analysis), calculate the lifts between these attributes and any 10 beers in your data. Choose one beer, and find the most similar beer (among the remaining 9) using the lift values. Explain your method. 

In [ ]:
word_list = {}
i = 0
for review in df['clean_review']:
    for word in review.split():
        if word not in word_list:
            word_list[word] = i
            i += 1

for word in attributes:
    if word not in word_list:
        word_list[word] = i
        i += 1

top_4_attributes = ['dark','sweet','carbonation','head']
for idx in range(0,4):
    for beer in (df['Beer']).unique():
        beer_df = df[df['Beer'] == beer]
        N = len(df)
        AB = sum(beer_df['clean_review'].str.contains(top_4_attributes[idx]))
        A = word_list[top_4_attributes[idx]]
        B = len(beer_df)
        lift = (N*AB)/(A*B)
        df.loc[df['Beer'] == beer, 'lift_{}'.format(top_4_attributes[idx])] = lift

beer_lifts = df.groupby('Beer').agg({'lift_{}'.format(top_4_attributes[0]): 'mean', 'lift_{}'.format(top_4_attributes[1]): 'mean', 'lift_{}'.format(top_4_attributes[2]): 'mean', 'lift_{}'.format(top_4_attributes[3]): 'mean'}).reset_index()[:10]

beer_lifts

In [ ]:
def cosine_similarity(vec1, vec2):
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))
desired_beer = '4th Anniversary'
#get the attributes columns for desired beer
desired_beer_attributes = beer_lifts[['lift_{}'.format(top_4_attributes[0]),'lift_{}'.format(top_4_attributes[1]),'lift_{}'.format(top_4_attributes[2]),'lift_{}'.format(top_4_attributes[3])]][beer_lifts['Beer'] == desired_beer]
#for each row in beer_lifts
for idx in range(0, len(beer_lifts)):
    current_beer_attributes = beer_lifts[['lift_{}'.format(top_4_attributes[0]),'lift_{}'.format(top_4_attributes[1]),'lift_{}'.format(top_4_attributes[2]),'lift_{}'.format(top_4_attributes[3])]].iloc[idx]
    beer_lifts.loc[idx, 'cosine_similarity'] = cosine_similarity(desired_beer_attributes, current_beer_attributes)
beer_lifts.sort_values(by='cosine_similarity', ascending=False)
beer_lifts.head(2)